<a href="https://colab.research.google.com/github/deepakawl/supplychain-analytics-teaching/blob/main/Alko_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Alko case, 46-893 Operations and Supply Chain Analytics, CMU-Tepper School of Business

In [1]:
import numpy as np
import pandas as pd
import math
import scipy.stats as st
from scipy.optimize import minimize

# Load Excel data
xls = pd.ExcelFile("Alko_students.xlsx")
df = xls.parse('Template')

# Extract demand data
region_blocks = [9, 16, 23, 30, 37]  # starting rows of demand blocks
product_ids = [1, 3, 7]  # 3 parts to analyze

avg_demand_matrix = np.zeros((3, 5))
std_demand_matrix = np.zeros((3, 5))
num_demand_matrix = np.zeros((3, 5))

for j, start_row in enumerate(region_blocks):
    block = df.iloc[start_row:start_row+3, [2, 3, 4]].dropna()
    block.columns = ['Part', 'Mean', 'Std dev']
    for i, pid in enumerate(product_ids):
        match = block[block['Part'] == pid]
        if not match.empty:
            avg_demand_matrix[i, j] = match['Mean'].values[0]
            std_demand_matrix[i, j] = match['Std dev'].values[0]
            num_demand_matrix[i, j] = pid  # simply treat Part ID as a proxy for units/year

# Parameters
days = 365
h = 0.15
tc = 0.19
ctc = 0.29
L = 5
T = 6
csl = 0.95

# Optimization function
def compute_cost(corr, avg_demand, std_demand, num_demand):
    n_parts, n_regions = avg_demand.shape
    corr_matrix = np.full((n_regions, n_regions), corr)
    np.fill_diagonal(corr_matrix, 1)

    def objective_function(flat_matrix):
        centralize = flat_matrix.reshape((n_parts, n_regions))
        agg_std = np.diag(np.dot(np.dot(np.multiply(std_demand, centralize), corr_matrix),
                                 np.transpose(np.multiply(std_demand, centralize))))
        sqrt_agg_std = np.sqrt(agg_std)
        sum_cols = np.sum(centralize, axis=1, keepdims=True)
        sum_cols[sum_cols == 0] = 1
        inv_sum_cols = 1 / sum_cols
        sqrt_agg_std2 = np.repeat(sqrt_agg_std.reshape(n_parts, 1), n_regions, axis=1)
        inv_sum_cols2 = np.repeat(inv_sum_cols.reshape(n_parts, 1), n_regions, axis=1)
        updated_std = centralize * inv_sum_cols2 * sqrt_agg_std2 + (1 - centralize) * std_demand

        total_cost = sum(
            days * num_demand[i, j] *
            (h * (st.norm.ppf(csl) * updated_std[i, j] * math.sqrt(T + L) +
                  avg_demand[i, j] * (T/2 + L)) +
             (avg_demand[i, j] * (centralize[i, j] * ctc + (1 - centralize[i, j]) * tc))
        ) for i in range(n_parts) for j in range(n_regions))

        return total_cost

    initial_guess = np.full((n_parts * n_regions,), 0.5)
    bounds = [(0, 1)] * len(initial_guess)
    constraints = ({'type': 'ineq', 'fun': lambda x: [xi * (1 - xi) for xi in x]})

    result = minimize(objective_function, initial_guess, bounds=bounds, constraints=constraints)
    optimal_matrix = np.round(result.x.reshape((n_parts, n_regions)))
    centralized_cost = days * sum(
        num_demand[i, j] * avg_demand[i, j] * optimal_matrix[i, j]
        for i in range(n_parts)
        for j in range(n_regions)
    )
    investment = centralized_cost * 2 if centralized_cost <= 400000 else (800000 + (centralized_cost - 400000) * 1.5)
    return {
        "correlation": corr,
        "min_cost": result.fun,
        "cdc_cost": centralized_cost,
        "investment": investment,
        "matrix": optimal_matrix
    }

# Run for correlation levels
for corr_val in [0, 0.5, 1]:
    result = compute_cost(corr_val, avg_demand_matrix, std_demand_matrix, num_demand_matrix)
    print(f"\nCorrelation: {corr_val}")
    print(f"Minimum Cost      : ${result['min_cost']:,.2f}")
    print(f"CDC Cost          : ${result['cdc_cost']:,.2f}")
    print(f"Investment Needed : ${result['investment']:,.2f}")
    print("Centralization Matrix:")
    print(result["matrix"])



Correlation: 0
Minimum Cost      : $124,911.44
CDC Cost          : $66,725.65
Investment Needed : $133,451.30
Centralization Matrix:
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]

Correlation: 0.5
Minimum Cost      : $141,057.20
CDC Cost          : $53,775.45
Investment Needed : $107,550.90
Centralization Matrix:
[[0. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]

Correlation: 1
Minimum Cost      : $147,790.51
CDC Cost          : $8,252.65
Investment Needed : $16,505.30
Centralization Matrix:
[[0. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
